In [ ]:

import casadi
import numpy as np
import matplotlib.pyplot as plt
from functools import partial
from tqdm import tqdm
from SHARED.model import *
from matplotlib.lines import Line2D
from SHARED.display_trajectories import *
import time
import timeit
from numpy import asarray
from numpy import savetxt
from SHARED.setup import *

#### ODE Model

##### Params

##### Model

In [ ]:
T = (5) #Time horizon (hours)
N = (T*3600)//dT #Number of Control Actions
F,g = model_functions()
stochastic = True
num_simulations = 30 if stochastic else 1
save_data = True

# N = 1920

In [ ]:
print (N)

#### Optimal Control

Objective Functions: $\newline$
J1 = $$\min_{u(k),y(k)} \sum_{k = k_0}^{k_0 + N_p} (V(y(k),u(k),d(k)))= \sum_{k = k_0}^{k_0 + N_p} -q_{y_1} \cdot y_1(k) + q_{u_1}\cdot u_1(k) +  q_{u_2}\cdot u_2(k) + q_{u_3}\cdot u_3(k)$$
$$s.t.$$
$$x(k+1) = F(x(k),u(k),d(k),p), \quad y(k) = g(x(k),p)$$
$$u_{min} \le u(k) \le u_{max}, \quad |u(k) - u(k-1)| \le \delta u$$
$$y_{min}(k) \le y(k) \le y_{max}(k), \quad \forall k=k_0,\dots,k0+N_p $$
$$x(k_0) = x_0$$

In [ ]:
print (N)

In [ ]:
opti = casadi.Opti()
mpc_reward = partial(reward_function, return_type = "DM")



#State Variables
X = opti.variable(4,N+1)    #->Drymass              (kg/m2)
                            #->Indoor C02           (kg/m3)
                            #->Indoor Temp          (deg C)
                            #->Indoor Humidity      (kg/m3)
#Output Variables
Y  = opti.variable(4,N)     #->Drymass              (kg/m2)
                            #->Indoor C02           (ppm)
                            #->Indoor Temp          (deg C)
                            #->Indoor Humidity      (%)
#Disturbance Variables
D = opti.parameter(4,N)     #->Irradiance           (W/m2)
                            #->Outdoor C02          (kg/m3)
                            #->Outdoor Temp         (deg C)
                            #->Outdoor Humidity     (kg/m3)
#Control Variables
U = opti.variable(3,N)      #->C02 Injection        (mg/(m2.s))
                            #->Ventilation          (mm/s)
                            #->Heating              (W/m2)

P = opti.variable (4,N)     #Penalties for temp ub and lb, co2 ub and lb                  
                            
#Initial Values
px = opti.parameter(4,1)             #initial state
pu = opti.parameter(3,1)             #initial control action

# lam_g = opti.
# lamx = 



#Set Parameters
opti.set_value(D,casadi.DM.zeros(D.shape))
opti.set_value(px,x0)
opti.set_value(pu,u0)


#Cost Function
J = 0

#Set Constraints and Cost Function
for k in range(0,N):
    
    
    opti.subject_to(X[:,k+1] == F(X[:,k],U[:,k],D[:,k],nominal_params))                                #Dynamic Constraints
    opti.subject_to(Y[:,k] == g(X[:,k+1],nominal_params))                                                #Output  Constraints
    opti.subject_to(u_min <= (U[:,k] <= u_max))                                         #Input   Contraints
    
    #Linear penalty functions
    opti.subject_to(P[:,k] >= 0)
    opti.subject_to(P[0,k] >= pen_c02     * (C02_MIN_CONSTRAIN_MPC - Y[1,k]))   #C02 lb
    opti.subject_to(P[1,k] >= pen_c02     * (Y[1,k] - C02_MAX_CONSTRAIN_MPC))   #C02 ub
    opti.subject_to(P[2,k] >= pen_temp_lb * (TEMP_MIN_CONSTRAIN_MPC - Y[2,k]))             #Temp lb
    opti.subject_to(P[3,k] >= pen_temp_ub * (Y[2,k] - TEMP_MAX_CONSTRAIN_MPC))                  #Temp ub
    
         
    
    #Hard Constraints
    J -= mpc_reward(delta_drymass=(X[0,k+1]-X[0,k]), control_inputs=U[:,k])       
    # J -= mpc_reward(control_inputs=U[:,k])  
    J += (P[0,k] + P[1,k] + P[2,k] + P[3,k])    
    
    # opti.subject_to(C02_MIN_CONSTRAIN_MPC     <= (Y[1,k] <= C02_MAX_CONSTRAIN_MPC))     #C02 min max constraints
    # opti.subject_to(T_mins[0,k] <= (Y[2,k]    <= T_maxs[0,k])) #Temp min max constraints
    # opti.subject_to(HUM_MIN_CONSTRAIN         <= (Y[3,k] <= HUM_MAX_CONSTRAIN))         #Hum min max constraints
    
    if k < N-1:                                         
        opti.subject_to(-delta_u<=(U[:,k+1] - U[:,k]<=delta_u))               #Change in input Constraint

# J -= mpc_reward(delta_drymass=(X[0,N])) 
opti.subject_to(-delta_u <= (U[:,0]-pu <= delta_u))                             #Initial change in input Constraint
opti.subject_to(X[:,0] == px)                                                           #Initial Condition Constraint
opti.minimize(J)

#Solver --> ipopt, bonmin, snopt, blocksqp,sqpmethod, scpgen
opts = {}
opts["print_time"] = False
opts["ipopt.print_level"]= False
opts["verbose"] =  False
opts["ipopt.max_iter"] = 5000

# opts["ipopt.tol"] = 1 #Default: 0.0001
# opts["ipopt.constr_viol_tol"] = 1 #Default: 0.0001
# opts["ipopt.acceptable_constr_viol_tol"] = 1 #Default: 0.0001
# opts["ipopt.acceptable_tol"] = 1 #Default: 0.0001
# opts["ipopt.jac_d_constant"] = 'yes'
# opts["ipopt.acceptable_tol"] = 1e-5
opts["ipopt.bound_relax_factor"] = 1e-3

opts["ipopt.nlp_scaling_method"]       = 'gradient-based'
# opts["ipopt.warm_start_same_structure"]       = 'yes'
# opts["ipopt.warm_start_init_point"]    = 'yes'
opts["ipopt.warm_start_entire_iterate"]    = 'yes'
# opts["ipopt.bound_push"] = 0.5
# opts["ipopt.bound_frac"] = 0.5
# opts["ipopt.slack_bound_push"] = 0.5
# opts["ipopt.slack_bound_frac"] = 0.5
# opts["ipopt.max_cpu_time"] = 0.1


# s_opts = {"max_cpu_time": 0.1, 
# 				  "print_level": 0, 
# 				  "tol": 5e-1, 
# 				  "dual_inf_tol": 5.0, 
# 				  "constr_viol_tol": 1e-1,
# 				  "compl_inf_tol": 1e-1, 
# 				  "acceptable_tol": 1e-2, 
# 				  "acceptable_constr_viol_tol": 0.01, 
# 				  "acceptable_dual_inf_tol": 1e10,
# 				  "acceptable_compl_inf_tol": 0.01,
# 				  "acceptable_obj_change_tol": 1e20,
# 				  "diverging_iterates_tol": 1e20}
s_opts = { "constr_viol_tol": 0.01,
            "acceptable_constr_viol_tol": 0.1, 
            "acceptable_tol": 1e-2, 
}

opti.solver('ipopt',opts,s_opts,)
# sol = opti.solve()
MPC = opti.to_function('MPC',[px,D,pu],[U[:,0],U,X],['px','D','pu'],['u_opt','u_traj','x_traj'])
MPC.save('MPC/MPC_controller_'+str(T) + 'hr')

# print (opti.debug.value(J))

# print (MPC([0.0025,0.001,15,0.008],np.zeros((4,12)),[0,0,50],10*np.ones((1,12)),20*np.ones((1,12))))
# MPC =casadi.Function.load('MPC.casadi')
# casadi.DM.set_precision(15)

#### Simulation

In [ ]:
get_d = partial(get_disturbance,weather_data = weather_data,start_time=start_time,Np=N, dt=dT)
    


for sim_num in range(num_simulations):
    print (f"Sim {sim_num}")
    
    #Save directories
    file_path = "Stochastic/" + str (T) + '_hr/scale_'+str(noise_scale) if stochastic else "Deterministic/" + str (T)
    directory = 'results/MPC/' + file_path + '/Sim_' + str(sim_num) + '/'
    if save_data:
        os.makedirs(directory, exist_ok=True)
   
    #Initial Conditions
    x = x0
    u_opt = u0
    


    #Reward evaluation function
    evaluate_rewards = partial(reward_evaluation,
                        constraint_mins=np.array([C02_MIN_CONSTRAIN_MPC, TEMP_MIN_CONSTRAIN_MPC, HUM_MIN_CONSTRAIN]),
                        constraint_maxs=np.array([C02_MAX_CONSTRAIN_MPC, TEMP_MAX_CONSTRAIN_MPC, HUM_MAX_CONSTRAIN]))



    #Logging states, outputs, disturbances and control inputs
    X_log, D_log, Y_log, U_log = [],[],[],[]


    #Logging reward and penalties
    cum_reward_log = [0]
    cum_penalties = [0]
    x_traj = np.zeros((4,N+1))
    u_traj = np.zeros((3,N))
    total_reward=0
    total_penalty=0

    #Logging compute time for control action
    comp_time_log = []
    for k in tqdm(range(0,max_steps)):
        
        if N == max_steps:
            
            if k == 0:
                print ("Getting optimal trajectory")
                timer = time.perf_counter()
                u_opt,u_traj,x_traj = MPC(x,d,u_opt)
                
                u_opt  = np.array(u_opt)      
                u_traj = np.array(u_traj)
                x_traj = np.array(x_traj)
                
                timer = time.perf_counter() - timer
                print ("Finished Optimizing")
              
            d_now  = d[:,k]
            u_opt = u_traj[:,k]
        else:
            d = get_d(k)  
            d_now  = d[:,0]
            timer = time.perf_counter()
            
            u_traj = np.roll(u_traj,shift=-1,axis=1)
            u_traj[:,-1] = np.copy(u_traj[:,-2])
            
            x_traj = np.roll(x_traj,shift=-1,axis=1)
            x_traj[:,-1] = np.copy(x_traj[:,-2])
            
            u_opt,u_traj,x_traj = MPC(x,d,u_opt)
            u_opt  = np.array(u_opt)      
            u_traj = np.array(u_traj)
            x_traj = np.array(x_traj)
            u_opt = np.clip(u_opt,u_min.reshape(3,1),u_max.reshape(3,1))
            timer = time.perf_counter() - timer
            
        #Evolve State
        sys_params = noisy.parametric_uncertainty() if stochastic else nominal_params
        x_next = F(x,u_opt,d_now,sys_params).toarray().ravel()
        # x_next = noisy.add_noise(x,x_next) if stochastic else x_next
        y = g(x_next,sys_params).toarray().ravel()
        
        #Reward Evaluation
        reward, penalties = evaluate_rewards(delta_drymass=x_next[0] - x[0],control_inputs=u_opt,
                    outputs2constrain=y[1:],)

        #Store Rewards and penalties recieved
        rr = reward-penalties
        
        total_reward += reward
        total_penalty += penalties
        cum_reward_log.append(cum_reward_log[-1] + reward-penalties)
        cum_penalties.append(cum_penalties[-1] + penalties)
        
    
        #Log Values
        X_log.append(x_next)
        D_log.append(d_now)
        Y_log.append(y)
        U_log.append(u_opt)
        comp_time_log.append(timer)
        
        #Repeat
        x=x_next
        
    # #Reshaping arrays
    U_log = np.array(U_log)
    Y_log = np.array(Y_log)
    D_log = np.array(D_log)
    comp_time_log =np.array(comp_time_log)
    
    if save_data:
        savetxt(os.path.join(directory, 'Y_log.csv'), Y_log[1:], delimiter=',')
        savetxt(os.path.join(directory, 'U_log.csv'), U_log[:,:,0], delimiter=',')
        savetxt(os.path.join(directory, 'D_log.csv'), D_log, delimiter=',')
        savetxt(os.path.join(directory, 'comp_time_log.csv'), comp_time_log, delimiter=',')
        savetxt(os.path.join(directory, 'rewards_log.csv'), np.array(cum_reward_log), delimiter=',')
    
    
    

##### Plotting Results

In [ ]:
#Printing metrics
print (f"Total Reward/EPI: {total_reward}")#this should be the same or very similar to epi
print (f"Total Penalty: {total_penalty}")
print_metrics(Y_log[:-1], U_log, D_log, day_range=(0,40), rewards=np.array(cum_reward_log), time_log=comp_time_log)